In [2]:
#System Imports
import os
import sys

from github import Github

# for requests funtion for github api
import requests
import json

from statistics import mean
import numpy as np
import pandas as pd


from bs4 import BeautifulSoup


import matplotlib.pyplot as plt
import scipy
from scipy.stats import ttest_ind
%matplotlib inline

import re
import json

import random

In [3]:
#2 github Repo Statistics
#mean is calculated if there is only that languagge in the file


# CLOC is used to get lines of code - https://github.com/AlDanial/cloc can be used here
# @software{adanial_cloc,
#   author       = {Albert Danial},
#   title        = {cloc: v1.92},
#   month        = dec,
#   year         = 2021,
#   publisher    = {Zenodo},
#   version      = {v1.92},
#   doi          = {10.5281/zenodo.5760077},
#   url          = {https://doi.org/10.5281/zenodo.5760077}
# }

In [4]:
# get github api token key
# will need to generate your own from github account
#file is in the same directory as notebook
token = open("token.txt", "r").read()
User = "Kaggle"


In [5]:
# get the number of enviroments
# uses an api call to get the json data from Github api
def get_enviroment(name, repo, token):
    headers = {"Accept": "application/vnd.github+json", "Authorization": f"Bearer {token}", "X-GitHub-Api-Version" : "2022-11-28"}
    request = requests.get(f'https://api.github.com/repos/{name}/{repo}/environments',  headers=headers)
#     print(request.json())
    return int(request.json()['total_count'])


# functions to pull individual stats
def get_contributors(repo):
    return len(list(repo.get_contributors(anon="true")))
def get_stars(repo):
    return repo.stargazers_count
def get_commits(repo):
    return len(list(repo.get_commits()))  
def get_branches(repo):
    return len(list(repo.get_branches()))
def get_tags(repo):
    return len(list(repo.get_tags()))
def get_forks(repo):
    return len(list(repo.get_forks()))
def get_releases(repo):
    return len(list(repo.get_releases()))
def get_closed(repo):
    return len(list(repo.get_issues(state='closed')))

# gets a list of all the repos 
def get_repos(token, User):
    gh = Github(token)
    repos = gh.get_user(User).get_repos()
    return repos

def get_names(repos):
    return [i.name for i in repos]
def generate_links(user, names):
    base = "https://github.com/"
    return [f'{base}/{user}/{name}' for name in names]
    
#download a repo without getting large files
def clone_repo(repo_link, repo_title,folder_dir):
    os.system(f"git lfs clone {repo_link} ./{folder_dir}/{repo_title}/ -X *.*")
#download all repos of the given links
def download_all(links, titles, user):
    for i,link in enumerate(links):
        clone_repo(link,titles[i],user)
        
# make the paths for github to download
def get_paths(names,user):
    return [f'./{user}/{name}' for name in names]        

# get statistics about repo 
def github_stats(token, User):
    repos = get_repos(token, User)
    #lists for stats 
    # used as lists instead of directly summing for if other stats ar wanted later
    stats = {}
    stars = []
    contributors = []
    Branches = []
    tags = []
    commits = []
    forks = []
    releases = []
    closed = []
    environments = []
    for repo in repos:
        contributors.append(get_contributors(repo))
        stars.append(get_stars(repo))
        commits.append(get_commits(repo))
        Branches.append(get_branches(repo))
        tags.append(get_tags(repo))
        forks.append(get_forks(repo))
        releases.append(get_releases(repo))
        closed.append(get_closed(repo))
        environments.append(get_enviroment(User, repo.name, token))
    stats["Sum of Commits"]= sum(commits)
    stats["Median of commits"]= mean(commits)
    stats["stars"]= sum(stars)
    stats["contributors"]= sum(contributors)
    stats["Branches"]= sum(Branches)
    stats["tags"]= sum(tags)
    stats["forks"]= sum(forks)
    stats["releases"]= sum(releases)
    stats["closed"]= sum(closed)
    stats["environments"]= sum(environments)

    return stats

# uses Clock to anyalize the data as well as add the spaces analysis together
def get_stats(links):
    files = []
    loc = []
    files = []
    languages = [] 
    codes = {}
    stats = {}
    for i,path in enumerate(links):
        data = os.popen(f'cloc-1.98.exe {path}').read().splitlines()
        iterations = len(data)-3
        for i in range(4,iterations):
            section = data[i].split()
            codes[section[0]]=[]
            stats[section[0]] = []
    for i,path in enumerate(links):
        data = os.popen(f'cloc-1.98.exe {path}').read().splitlines()
        # miss the first 4 structure of the data 
        iterations = len(data)-3
        for i in range(4,iterations):
            section = data[i].split()
            codes[section[0]].append(float(section[-1]))
    for i in stats.keys():
        stats[i].append(mean(codes[i]))
        stats[i].append(sum(codes[i]))
    return stats

In [6]:
t = github_stats(token,User)

In [7]:
repos = get_repos(token, User)
names = get_names(repos)
links = generate_links(User, names)
paths = get_paths(names,User)

In [8]:
# downloads repo for stats
download_all(links,names, User)

In [9]:
link =get_stats(paths)

In [10]:
pd.DataFrame(link,index=['average','sum'])

,Markdown,YAML,Julia,Dockerfile,Bourne,Text,XML,Python,CSV,JSON,Jupyter,R,TypeScript,SVG,CSS,JavaScript,Java,HTML,TOML
average,187.5,403.0,70.0,59.0,224.7,7722.0,24317.0,7581.8,109471.333333,278.333333,3785.75,229.666667,1490.0,17.0,11.0,1368.0,1608.0,688.0,142.0
sum,1875.0,2418.0,70.0,413.0,2247.0,30888.0,24317.0,37909.0,328414.000000,1670.000000,15143.00,689.000000,2980.0,17.0,11.0,2736.0,1608.0,688.0,142.0


In [11]:
pd.DataFrame(t, index=[0])

,Sum of Commits,Median of commits,stars,contributors,Branches,tags,forks,releases,closed,environments
0,6607,600.636364,9008,306,157,325,2471,236,2741,0


In [12]:
## 3 Crawl Issue Report

In [13]:
# gets the web page
def SearchResponse(top):
    return requests.get(top)

# gets all links
# takes the most recent issue and generates a list from 1 to most recent
def get_links(links_response, baselink):
    soup = BeautifulSoup(links_response.text, 'html.parser')
    script_text = str(soup.select("script")[9])
    script_text = script_text.encode('ascii').decode('unicode_escape').replace('\\n', '').replace('\\', '').replace("<script>", '')
    titles= script_text.split('[')[5].split(']')[0].replace('"','').split(',')
    new = titles[0].split('-')[-1]
    return [f'{baselink}CAMEL-{i}' for i in range(1,int(new)+1)]


def get_data(issue_response,link):
    soup = BeautifulSoup(issue_response.text, 'html.parser')
    script_text = str(soup.select("script")[15])
    
#   first converts the unicode chars in the script to their values and removes extra formatting
#   removes the script tag as this will get in the way of beautiful soups data
    script_text = script_text.encode('ascii').decode('unicode_escape').replace('\\n', '').replace('\\', '').replace("<script>", '')
    ee = BeautifulSoup(script_text, 'html.parser')
    data_point = {}
    Title = soup.find(id='summary-val').contents[0]
    Type = soup.find(id='type-val').contents[2].strip() 
    Assignee = soup.find(id='assignee-val').text.strip() 
    reporter = soup.find(id='reporter-val').text.strip() 
     
    Description = soup.find(id='description-val')
    if Description:
        Description=Description.text
#     print(Description)
    created = soup.find(id='created-val').text.strip()  
    updated = soup.find(id='updated-val').text.strip()  
    status = soup.find(id='status-val').contents[1].text
    Priority = soup.find(id='priority-val').text.strip()  
    Resolved = None
    if status.lower() == "resolved":
        Resolved = soup.find(id='resolutiondate-val').text.strip()   
    else:
        Resolved = "open"

    comments = ee.findAll('div', id=lambda x: x and x.startswith('comment-'))
    delimitedComments =[]
    for i,comment in enumerate(comments):
        delimitedComments.append(comments[i].text)
    # stores the string of characthers into one text file in order to keep a 2d list. Delimiter made to split again is $_$ to be unique
    delimitedComments = "$_$".join(delimitedComments)
    
    data_point['Title']=Title
    data_point['Type']=Type
    data_point['Assignee']=Assignee
    data_point['reporter']=reporter
    data_point['Description']=Description
    data_point['created']=created
    data_point['updated']=updated
    data_point['status']=status
    data_point['Priority']=Priority
    data_point['Resolved']=Resolved
    data_point['link']=link
    if comments:
        data_point['comments']=delimitedComments 
    else:
        data_point['comments']=None 
    
    return data_point
    
        
def get_all_data(all_links):
    all_data = []
    for i in all_links:
        all_data.append(get_data(SearchResponse(i),i))
    return all_data    
    


In [14]:
#single line testing
# issue = 'https://issues.apache.org/jira/browse/CAMEL-10597'
# issue = 'https://issues.apache.org/jira/browse/CAMEL-11347'
# issue_response = SearchResponse(issue)


In [15]:
# base for generating all the links
baselink = 'https://issues.apache.org/jira/browse/'
# gets the most recent issue
domain = "https://issues.apache.org/jira/projects/CAMEL/issues/?filter=allissues"
links_response = SearchResponse(domain)


In [16]:
# get_data(issue_response,issue)

In [17]:
all_links =get_links(links_response, baselink)

In [18]:
# sample 50 entries to not get Jira angry
data = get_all_data(random.sample(all_links, 50))

In [19]:
df = pd.DataFrame(data)
df.to_csv("output.csv")